In [1]:
# For Data Generation
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import string

In [2]:
class DataGenerator:
    def __init__(self, rows = 20, cols_float = 2, cols_int = 2, cols_words=3, cols_group=3
                 ,numrange = 12, strrange = 500, draw = False, wordsCount=5, groupsElements=10, groupsWordsCount=2):
        self.rows = rows
        self.cols_float = cols_float
        self.cols_int = cols_int
        self.cols_words = cols_words
        self.numrange = numrange 
        self.strrange = strrange
        self.wordsCount = wordsCount
        self.cols_group = cols_group
        self.groupsWordsCount=groupsWordsCount
        self.groupsElements = groupsElements
        self.draw = draw
        #https://github.com/dwyl/english-words
        words_file="https://computersciencewiki.org/images/1/13/Words_alpha.txt"
        self.words = pd.read_csv(words_file, sep=" ", header=None)
    
    def set_rows(self, nrows):
      self.rows = nrows
      
    def print(self):
      print ("Rows: {}".format(self.rows))
      print ("Float Columns: {}".format(self.cols_float))
      print ("Integer Columns: {}".format(self.cols_int))
      # Add more as needed
          
    def generateData(self):
        df = pd.DataFrame()
        random.seed(1)
        
        for i in range(self.cols_float):
            col_name =  'float' + str(i)
            
            seed = random.randint(0,10000) 
            df[col_name] = self.lognuniform(use_int = False, seed = seed)
            if self.draw == True:
                self.plot_density(df[col_name])
                     
        for i in range(self.cols_int):
            col_name = 'int' + str(i)
            seed = random.randint(0,10000) 
            df[col_name] = self.lognuniform(use_int = True, seed = seed)
            if self.draw == True:
                self.plot_density(df[col_name])
        
        for i in range(self.cols_words):
            col_name = 'words' + str(i)
            df[col_name] = self.randomWords() #(N = n)
        
        for i in range(self.cols_group):
            col_name = 'group' + str(i)
            df[col_name] = self.randomGroups() #(N = n)
       
        return df         
    
    def lognuniform(self, base=np.e, use_int = True, seed = 0):
        np.random.seed(seed) 
        # Note: you can not use random.seed here since the np.random.uniform needs np.random.seed not random.seed.
        multiplier = np.random.choice([-1,1], size= self.rows)
        # -5 is used since we done want most of the value to be less than base
        exponentials = np.random.uniform(low = -5, high = self.numrange, size = self.rows)
        #data = np.power(base, np.random.uniform(low = -self.numrange, high = self.numrange, size = self.rows))
        data = np.power(base, exponentials)  * multiplier
        if (use_int == True):
            return data.astype(int)
        if (use_int == False):
            return data
    
    def randomWords(self):
        mylist = []
        for i in range(self.rows):
            mystring = (' '.join(str(self.words.iat[random.randint(0,self.words.shape[0]-1),0]) for i in range(random.randint(1,self.wordsCount)))).strip()
            mylist.append(mystring)
        return mylist
    
    def randomGroups(self):
        groups = []
        for i in range(self.groupsElements):
            mystring = (' '.join(str(self.words.iat[random.randint(0,self.words.shape[0]-1),0]) for i in range(random.randint(1,self.groupsWordsCount)))).strip()
            groups.append(mystring)
        mylist = []
        for i in range(self.rows):
            mystring = groups[random.randint(0,len(groups)-1)]
            mylist.append(mystring)
        return mylist
    
    def plot_density(self, col):
        count, bins, ignored = plt.hist(col, 15, density=True)
        plt.plot(bins, np.ones_like(bins), linewidth=2, color='r')
        plt.show()                

In [3]:
size = "500MB"

# Default 10MB file
datagen = DataGenerator(rows = 10000, cols_float = 20, cols_int = 20, cols_words=10, cols_group=11, wordsCount = 10)

if size == "100MB":
  datagen.set_rows(100000) 
if size == "500MB":
  datagen.set_rows(500000) 
if size == "1GB":
  datagen.set_rows(1000000) 

In [4]:
df = datagen.generateData()

In [5]:
file_loc = "/dbfs"
file_prefix = "/mnt/blob/datasets/"
file_name = "dataset_" + size

In [6]:
# Make directory if it does not exist
dbutils.fs.mkdirs(file_prefix)

# Remove existing files for new run
dbutils.fs.rm(file_prefix + file_name + ".csv")

# Write new file (needs /dbfs added to the begining of file name)
df.to_csv(file_loc + file_prefix + file_name + ".csv", index = False)

In [7]:
# Writing 100 new rows
datagen.set_rows(100) 
datagen.print()
data2 = datagen.generateData()
data2.shape

# Remove existing files for new run
dbutils.fs.rm(file_prefix + "add_100.csv")
# Write new file (needs /dbfs added to the begining of file name)
data2.to_csv(file_loc + file_prefix + file_name + "_add_100.csv", index = False)

Rows: 100
Float Columns: 20
Integer Columns: 20

In [8]:
# Writing 1000 new rows
datagen.set_rows(1000) 
datagen.print()
data2 = datagen.generateData()
data2.shape

# Remove existing files for new run
dbutils.fs.rm(file_prefix + "add_1000.csv")
# Write new file (needs /dbfs added to the begining of file name)
data2.to_csv(file_loc + file_prefix + file_name + "_add_1000.csv", index = False)

Rows: 1000
Float Columns: 20
Integer Columns: 20

In [9]:
# Writing 10000 new rows
datagen.set_rows(10000) 
datagen.print()
data2 = datagen.generateData()
data2.shape

# Remove existing files for new run
dbutils.fs.rm(file_prefix + "add_10000.csv")
# Write new file (needs /dbfs added to the begining of file name)
data2.to_csv(file_loc + file_prefix + file_name + "_add_10000.csv", index = False)

Rows: 10000
Float Columns: 20
Integer Columns: 20

In [10]:
# Remove all files in location (use with caution)
#%fs rm -r /mnt/blob/